# I/O

In [1]:
import pandas as pd
pd.set_option("display.max_rows", 5)

In [2]:
pd.DataFrame({'Yes': [50, 21], 'No': [131, 2]})

,No,Yes
0,131,50
1,2,21


In [3]:
pd.DataFrame({'Bob': ['I liked it.', 'It was awful.'],
              'Sue': ['Pretty good.', 'Bland.']},
             index=['Product A', 'Product B'])

,Bob,Sue
Product A,I liked it.,Pretty good.
Product B,It was awful.,Bland.


Serie: only rhe list is mandatory. iF a Dataframa is like a table a Series is like a column

In [4]:
pd.Series([30, 35, 40], index=['2015 Sales', '2016 Sales', '2017 Sales'], name='Product A')

2015 Sales    30
2016 Sales    35
2017 Sales    40
Name: Product A, dtype: int64

## CSV

In [5]:
reviews = pd.read_csv("./data/winemag.csv", index_col=0)  # index is optional

In [6]:
reviews.shape  # (num entries, num cols)

(129971, 13)

reviews.head()

In [7]:
#wine_reviews.head().to_csv("wine_reviews.csv")  # To save data back

##  Excel

``` py
wic = pd.read_excel("../input/publicassistance/xls_files_all/WICAgencies2013ytd.xls", sheet_name='Total Women')

wic.to_excel('wic.xlsx', sheet_name='Total Women')
```

## SQL 

``` python
import sqlite3
conn = sqlite3.connect("../input/188-million-us-wildfires/FPA_FOD_20170508.sqlite")
fires = pd.read_sql_query("SELECT * FROM fires", conn)
ires.head()

conn = sqlite3.connect("fires.sqlite")
fires.head(10).to_sql("fires", conn)
```

# Indexing, selectin, assigning reference


In [8]:

reviews = pd.read_csv("./data/winemag.csv", index_col=0)

In [9]:
reviews.country  # From DataFrame return a column as a Serie
reviews["country"]  # Same but programaticallly

0            Italy
1         Portugal
            ...   
129969      France
129970      France
Name: country, Length: 129971, dtype: object

In [10]:
reviews["country"][1]  # Works like a dictionary (row) and list (col)

'Portugal'

## iloc

The first inex correspond to a row, the second one to a col (just the oposite as pthon)
It admite slices, index and lists, included negative numbers.
Indexing [a, b)

In [11]:
reviews.iloc[0:3, 0][1]

'Portugal'

In [12]:
reviews.iloc[[0, 1, 2], -2]

0       White Blend
1    Portuguese Red
2        Pinot Gris
Name: variety, dtype: object

## loc

Used to acces by label.
Other things like iloc
Indexing [a, b]

In [13]:
reviews.loc[1, 'country']

'Portugal'

In [14]:
reviews.loc[:, ['taster_name', 'taster_twitter_handle', 'points']]

,taster_name,taster_twitter_handle,points
0,Kerin O’Keefe,@kerinokeefe,87
1,Roger Voss,@vossroger,87
...,...,...,...
129969,Roger Voss,@vossroger,90
129970,Roger Voss,@vossroger,90


In [15]:
reviews.loc['Apples':'Potatoes']

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery


### Index
The index to use when selecting data can be changed aribtrally:

In [16]:
df = reviews.set_index("country")  # Returns a new dataframe object

## Conditional selection

In [17]:
reviews.country == 'Spain'  # This return a boolen Seties containing wether or not an item fulfills the condition
reviews.loc[reviews.country.isin(['Italy', 'France'])]
reviews.loc[reviews.price.notnull()]
reviews.loc[(reviews.country == 'Spain') & (reviews.points >= 98)]  # | for or


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
40307,Spain,Luscious prune and blackberry aromas come with...,Clon de la Familia,98,450.0,Northern Spain,Ribera del Duero,NaN,Michael Schachner,@wineschach,Emilio Moro 2009 Clon de la Familia (Ribera d...,Tinto Fino,Emilio Moro


### Assigning data

Just do it like any var

# Summary functions and maps


In [18]:
reviews.variety.describe()

count         129970
unique           707
top       Pinot Noir
freq           13272
Name: variety, dtype: object

In [19]:
reviews.points.mean()



88.44713820775404

In [20]:
reviews.country.unique()

array(['Italy', 'Portugal', 'US', 'Spain', 'France', 'Germany',
       'Argentina', 'Chile', 'Australia', 'Austria', 'South Africa',
       'New Zealand', 'Israel', 'Hungary', 'Greece', 'Romania', 'Mexico',
       'Canada', nan, 'Turkey', 'Czech Republic', 'Slovenia', 'Luxembourg',
       'Croatia', 'Georgia', 'Uruguay', 'England', 'Lebanon', 'Serbia',
       'Brazil', 'Moldova', 'Morocco', 'Peru', 'India', 'Bulgaria',
       'Cyprus', 'Armenia', 'Switzerland', 'Bosnia and Herzegovina',
       'Ukraine', 'Slovakia', 'Macedonia', 'China', 'Egypt'], dtype=object)

In [21]:
reviews.country.value_counts()

US        54504
France    22093
          ...  
Egypt         1
China         1
Name: country, Length: 43, dtype: int64

In [22]:
review_points_mean = reviews.points.mean()
reviews.points.map(lambda p: p - review_points_mean)  # Map -> Series

0        -1.447138
1        -1.447138
            ...   
129969    1.552862
129970    1.552862
Name: points, Length: 129971, dtype: float64

In [23]:
def remean_points(srs):
    srs.points = srs.points - review_points_mean
    return srs

reviews.apply(remean_points, axis='columns')  # Apply -> DataFRame

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,-1.447138,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,-1.447138,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,1.552862,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss
129970,France,"Big, rich and off-dry, this is powered by inte...",Lieu-dit Harth Cuvée Caroline,1.552862,21.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Schoffit 2012 Lieu-dit Harth Cuvée Car...,Gewürztraminer,Domaine Schoffit


Pandas can dededuce basic operation meaning

In [24]:
reviews.points - review_points_mean
reviews.country + " - " + reviews.region_1

0            Italy - Etna
1                     NaN
               ...       
129969    France - Alsace
129970    France - Alsace
Length: 129971, dtype: object

`idmax()` returns the index of the first index of a maximun value

In [25]:
reviews.iloc[(reviews.points / reviews.price).idxmax()].price

4.0

# Grouping and sorting reference


In [26]:
reviews.groupby('points').points.min()

points
80      80
81      81
      ... 
99      99
100    100
Name: points, Length: 21, dtype: int64

Once is grouped we can process it as a DF od DF

In [27]:
reviews.groupby(['winery']).apply(lambda df: df.title.iloc[0])

winery
1+1=3                          1+1=3 NV Rosé Sparkling (Cava)
10 Knots                 10 Knots 2010 Viognier (Paso Robles)
                                  ...                        
àMaurice    àMaurice 2013 Fred Estate Syrah (Walla Walla V...
Štoka                         Štoka 2009 Izbrani Teran (Kras)
Length: 16757, dtype: object

In [28]:
# Best wine of eacj province of Spain
reviews.loc[reviews.country == "Spain"].groupby(['country', 'province']).apply(lambda df: df.loc[df.points.argmax()])

country  \
country province                  
Spain   Andalucia         Spain   
        Catalonia         Spain   
...                         ...   
        Spain Other       Spain   
        Spanish Islands   Spain   

                                                               description  \
country province                                                             
Spain   Andalucia        One of the world's best PX wines is Solera 192...   
        Catalonia        This single-vineyard Carignan is living proof ...   
...                                                                    ...   
        Spain Other      This muscular, stout Monastrell sweet wine is ...   
        Spanish Islands  With so few Mallorcan wines on the market, Ani...   

                                     designation  points  price  \
country province                                                  
Spain   Andalucia                    Solera 1927      96   30.0   
        Catalonia                       El Perer      96  770.0   
...                                          ...     ...    ...   
        Spain Other      Recóndita Armonía Dulce      94   35.0   
        Spanish Islands                       Àn      92   50.0   

                                province                 region_1  region_2  \
country province                                                              
Spain   Andalucia              Andalucia         Montilla-Moriles       NaN   
        Catalonia              Catalonia                  Priorat       NaN   
...                                  ...                      ...       ...   
        Spain Other          Spain Other                    Spain       NaN   
        Spanish Islands  Spanish Islands  Vi de la Terra Mallorca       NaN   

                               taster_name taster_twitter_handle  \
country province                                                   
Spain   Andalucia        Michael Schachner           @wineschach   
        Catalonia        Michael Schachner           @wineschach   
...                                    ...                   ...   
        Spain Other      Michael Schachner           @wineschach   
        Spanish Islands  Michael Schachner           @wineschach   

                                                                     title  \
country province                                                             
Spain   Andalucia        Alvear NV Solera 1927 Pedro Ximénez (Montilla-...   
        Catalonia            Marco Abella 2012 El Perer Carignan (Priorat)   
...                                                                    ...   
        Spain Other      Bodegas Gutiérrez de la Vega 2013 Recóndita Ar...   
        Spanish Islands  Ànima Negra 2011 Àn Red (Vi de la Terra Mallorca)   

                               variety                        winery  
country province                                                      
Spain   Andalucia        Pedro Ximénez                        Alvear  
        Catalonia             Carignan                  Marco Abella  
...                                ...                           ...  
        Spain Other         Monastrell  Bodegas Gutiérrez de la Vega  
        Spanish Islands      Red Blend                   Ànima Negra  

[8 rows x 13 columns]

In [29]:
countries_reviewed = reviews.groupby(['country', 'province']).description.agg([len])
countries_reviewed = countries_reviewed.reset_index()
countries_reviewed.sort_index()  # Default
countries_reviewed.sort_values(by=['len'],  ascending=False)

,country,province,len
392,US,California,36247
415,US,Washington,8639
...,...,...,...
63,Chile,Coelemu,1
149,Greece,Beotia,1


In [30]:
display(reviews)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss
129970,France,"Big, rich and off-dry, this is powered by inte...",Lieu-dit Harth Cuvée Caroline,90,21.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Schoffit 2012 Lieu-dit Harth Cuvée Car...,Gewürztraminer,Domaine Schoffit
